## Proceso de Extracción, Transformación y Carga de los Datos basado en el Análisis Exploratorio de los Datos 

### Se importan las librerías a utilizar.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd

### Proceso de Web Scraping para obtener los datasets desde la Página Web del Observatorio de Movilidad y Seguridad Vial (OMSV)

In [2]:
# URL de la página que contiene el enlace al archivo que deseas descargar
url_pagina = 'https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales'

# Realizar la solicitud GET a la página web
response = requests.get(url_pagina)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Analizar el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar el segundo contenedor con la clase 'pkg-container'
    contenedor = soup.find('div', {'id': 'pkg-main-info'}) \
                    .find_all('div', {'class': 'pkg-container'})[1]

    # Buscar el enlace dentro de 'pkg-actions'
    pkg_actions = contenedor.find('div', {'class': 'pkg-actions'})
    enlace_archivo = pkg_actions.find('a', {'class': 'btn'})

    if enlace_archivo:
        # Obtener el valor del atributo href
        url_descarga = enlace_archivo.get('href')

        print('Enlace del archivo:', url_descarga)
    else:
        print('No se encontró el enlace al archivo en el segundo contenedor.')

else:
    print('Error al realizar la solicitud HTTP. Código de estado:', response.status_code)

Enlace del archivo: https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/victimas-siniestros-viales/homicidios.xlsx


### Se descarga el archivo desde la URL especificada y se guarda localmente en el archivo 'datasets/hechos.xlsx'. Si la descarga es exitosa, muestra un mensaje de éxito; de lo contrario, muestra un mensaje de error con el código de estado.

In [3]:
response = requests.get(url_descarga)

# Verifica si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Guarda el contenido del archivo en un archivo local
    with open('datasets/hechos.xlsx', 'wb') as f:
        f.write(response.content)

    print('Archivo descargado exitosamente.')
else:
    print(f'Error al descargar el archivo. Código de estado: {response.status_code}')

Archivo descargado exitosamente.


### Se leen todas las hojas del archivo HECHOS.xlsx, se guarda cada hoja como un archivo CSV en la carpeta datasets y se muestran mensajes de éxito para cada archivo creado. (Para trabajar con hojas de cálculo de Excel es necesario tener instalada la librería openpyxl)

In [4]:
# Especifica la ruta de tu archivo Excel
archivo_excel = 'datasets/hechos.xlsx'

# Lee todas las hojas del archivo Excel
hojas_excel = pd.read_excel(archivo_excel, sheet_name=None)

# Especifica la carpeta donde deseas guardar los archivos CSV
carpeta_csv = 'datasets/'

# Itera sobre cada hoja y guarda el DataFrame en un archivo CSV
for nombre_hoja, df in hojas_excel.items():
    # Genera el nombre del archivo CSV para cada hoja
    archivo_csv = f'{carpeta_csv}{nombre_hoja}.csv'

    # Guarda el DataFrame en formato CSV
    df.to_csv(archivo_csv, index=False)

    print(f'Archivo CSV "{archivo_csv}" creado exitosamente.')

Archivo CSV "datasets/HECHOS.csv" creado exitosamente.
Archivo CSV "datasets/DICCIONARIO_HECHOS.csv" creado exitosamente.
Archivo CSV "datasets/VICTIMAS.csv" creado exitosamente.
Archivo CSV "datasets/DICCIONARIO_VICTIMAS.csv" creado exitosamente.
Archivo CSV "datasets/clas.csv" creado exitosamente.


### Este proceso se realiza luego de la ejecucion de la primera fase del análisis exploratorio de los datos con la finalidad de buscar valores faltantes, valores atípicos u outliers y registros duplicados.

In [5]:
hechos = 'datasets\HECHOS.csv'
hechos_victimas= 'datasets\VICTIMAS.csv'

df_hechos_original = pd.read_csv(hechos, encoding=' UTF-8')
df_hechos_victimas_original = pd.read_csv(hechos_victimas, encoding='UTF-8')

### Se visualiza el DataFrame 'Hecho'

In [6]:
df_hechos_original.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


### Se obtienen los nombres de las columnas en el DataFrame

In [7]:
df_hechos_original.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

### Se eliminan columnas específicas del DataFrame con el objetivo de reducir su tamaño y mejorar su eficiencia.

In [8]:
df_hechos_modificado = df_hechos_original.drop(['AAAA', 'MM', 'DD', 'HORA' , 'XY (CABA)','PARTICIPANTES'],axis=1 )

### Se reemplazar ciertos valores NaN por 0.

In [9]:
df_hechos_modificado['N_VICTIMAS'] = df_hechos_modificado['N_VICTIMAS'].fillna(0)
df_hechos_modificado['COMUNA'] = df_hechos_modificado['COMUNA'].fillna(0)

### Se reemplazan valores float a números entero.

In [10]:
df_hechos_modificado['N_VICTIMAS'] = df_hechos_modificado['N_VICTIMAS'].map(int)
df_hechos_modificado['COMUNA'] = df_hechos_modificado['COMUNA'].map(int)

### Se eliminan filas con valores nulos

In [11]:
df_hechos_modificado = df_hechos_modificado.dropna(subset=['ID'])

### Se visualiza el DataFrame para confirmar si muestra los resultados deseados. 

In [12]:
df_hechos_modificado.head(5)

,ID,N_VICTIMAS,FECHA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,pos x,pos y,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,1,2016-01-02,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.50877521,-34.66977709,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,1,-58.39040293,-34.63189362,MOTO,AUTO
3,2016-0004,1,2016-01-10,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,-58.46503904,-34.68092974,MOTO,SD
4,2016-0005,1,2016-01-21,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,-58.38718297,-34.62246630,MOTO,PASAJEROS


### El DataFrame obtenido de los diferentes procesos se exporta a un archivo .CSV

In [13]:
df_hechos_modificado.to_csv('datasets/hechos_clean.csv', index=False)

### En este punto, se realiza el proceso de ETL al Dataset 'Víctima'.
### Se renombra la columna 'ID_hecho' por 'ID'.

In [14]:
df_hechos_victimas_modificado = df_hechos_victimas_original.rename(columns={'ID_hecho': 'ID'})

### Se eliminan columnas ['AAAA', 'MM', 'DD'] del DataFrame victimas para tomar solo la columna ['FECHA']

In [15]:
df_hechos_victimas_modificado_1 = df_hechos_victimas_modificado.drop([ 'AAAA', 'MM', 'DD'],axis=1 )

### Se visualiza el DataFrame 'Victimas'

In [16]:
df_hechos_victimas_modificado_1.head(5)

,ID,FECHA,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


### El DataFrame obtenido de los diferentes procesos se exporta a un archivo .CSV

In [17]:
df_hechos_victimas_modificado_1.to_csv('datasets/victimas_clean.csv', index=False )

### Se combinan ambos DataFrame 'Hechos' y 'Víctimas' para obtener solo uno. 

In [18]:
df_merged = pd.merge(df_hechos_modificado, df_hechos_victimas_modificado_1, on='ID', how='inner')

### Se visualiza el DataFrame obtenido de la unión de ambos. 

In [19]:
df_merged.head(5)

,ID,N_VICTIMAS,FECHA_x,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,...,pos x,pos y,VICTIMA_x,ACUSADO,FECHA_y,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",...,-58.47533969,-34.68757022,MOTO,AUTO,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",...,-58.50877521,-34.66977709,AUTO,PASAJEROS,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,...,-58.39040293,-34.63189362,MOTO,AUTO,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",...,-58.46503904,-34.68092974,MOTO,SD,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,1,2016-01-21,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",...,-58.38718297,-34.62246630,MOTO,PASAJEROS,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


### Se obtienen los nombres de las columnas en el DataFrame

In [20]:
df_merged.columns

Index(['ID', 'N_VICTIMAS', 'FECHA_x', 'HH', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE',
       'Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'COMUNA', 'pos x',
       'pos y', 'VICTIMA_x', 'ACUSADO', 'FECHA_y', 'ROL', 'VICTIMA_y', 'SEXO',
       'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')

### Se eliminan columnas que contienen la misma información en el DataFrame.

In [21]:
df_merged_clean = df_merged.drop(['FECHA_y', 'VICTIMA_y' ], axis = 1)

### El DataFrame obtenido se exporta a un archivo .CSV

In [22]:
df_merged_clean.to_csv('datasets/merged_clean.csv', index=False)

### El Dataset obtenido se carga con la función de pandas read_csv() para realizar las últimas transformaciones en los datos. 

In [23]:
final='datasets/merged_clean.csv'
df_final = pd.read_csv(final, encoding='UTF-8')

In [24]:
df_final.head(5)

,ID,N_VICTIMAS,FECHA_x,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,pos x,pos y,VICTIMA_x,ACUSADO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.47533969,-34.68757022,MOTO,AUTO,CONDUCTOR,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.50877521,-34.66977709,AUTO,PASAJEROS,CONDUCTOR,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,1,-58.39040293,-34.63189362,MOTO,AUTO,CONDUCTOR,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,-58.46503904,-34.68092974,MOTO,SD,CONDUCTOR,MASCULINO,18,SD
4,2016-0005,1,2016-01-21,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,-58.38718297,-34.62246630,MOTO,PASAJEROS,CONDUCTOR,MASCULINO,29,2016-02-01 00:00:00


### Se elimina la columna 'Altura' ya que tiene más del 82% de datos faltantes en sus registros. 

In [25]:
df_final = df_final.drop(columns=['Altura'])

### Se elimina un registro de la columna 'Calle' por estar completamente vacío.

In [26]:
df_final.dropna(subset=['Calle'], inplace=True)

### Se reemplazan valores erroneos en las columnas 'VICTIMA_x' y en la columna 'EDAD'

In [27]:
df_final['VICTIMA_x'] = df_final['VICTIMA_x'].replace(['PEATON MOTO'], 'SD')

In [35]:
df_final['EDAD'] = df_final['EDAD'].replace(['OBJETO FIJO'], '')

### Se verifica el tipo de datos del DataFrame resultante. 

In [40]:
df_final.dtypes

ID                       object
N_VICTIMAS                int64
FECHA_x                  object
HH                       object
LUGAR_DEL_HECHO          object
TIPO_DE_CALLE            object
Calle                    object
Cruce                    object
Dirección Normalizada    object
COMUNA                    int64
pos x                    object
pos y                    object
VICTIMA_x                object
ACUSADO                  object
ROL                      object
SEXO                     object
EDAD                     object
FECHA_FALLECIMIENTO      object
dtype: object

### El DataFrame obtenido se exporta a un archivo .CSV

In [41]:
df_final.to_csv('DataSetFinal.csv', index=False)